This kernel is based primarily on:

- Competition: https://www.kaggle.com/c/siim-covid19-detection/overview
- Starter kernel: https://www.kaggle.com/ayuraj/train-covid-19-detection-using-yolov5
- Primary dataset: https://www.kaggle.com/c/siim-covid19-detection/data
- Resized dataset: https://www.kaggle.com/xhlulu/siim-covid19-resized-to-1024px-jpg (others [here](https://www.kaggle.com/c/siim-covid19-detection/discussion/239918))
- Detection model: YOLOv5 - https://github.com/ultralytics/yolov5
- Tracking: Weights and Biases (integrated with YOLOv5)

Notes:

- RUN THE EXPERIMENT ON A GPU INSTANCE! This is going to take a sweet, sweet while either way.

# Setup and Variables

In [ ]:
# Are we training on the original dataset (and thus need to resize it, etc)?
RUN_ON_ORIGINAL = True

# Run only inference tasks? (You still need the separate noteboook for now.)
INFERENCE_ONLY = True

YOLOV5_REPO = '/kaggle/input/ultralyticsyolov5a'
KAGGLE_DATASET = '/kaggle/input/siim-covid19-detection'

PROJECT_NAME = 'kaggle-siim-covid'
EXP_NAME = 'exp'

WEIGHTS_FILE = 'best.pt'
IMG_SIZE = 1024
BATCH_SIZE = 16
EPOCHS = 10

# Pick the data source to train on:
KAGGLE_RESIZED = '/kaggle/tmp' if RUN_ON_ORIGINAL else f'/kaggle/input/siim-covid19-resized-to-{IMG_SIZE}px-jpg'
TRAIN_PATH = KAGGLE_RESIZED + '/train/'
TEST_PATH = KAGGLE_RESIZED + '/test/'

# Path to the final inference model:
MODEL_PATH = f'{PROJECT_NAME}/{EXP_NAME}/weights/{WEIGHTS_FILE}'

In [ ]:
%cd /kaggle/working

# GDCM:
!cp /kaggle/input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2

# YOLOv5:
!cp -r {YOLOV5_REPO} yolov5

# YOLOv5 default weights:
#!cp /kaggle/input/ultralyticsyolov5aweights/* yolov5/

# If you want to supply your own pretrained model, dump it into a
# dataset and copy it over to the YOLOv5 directory here.
!mkdir -p yolov5/{PROJECT_NAME}/{EXP_NAME}/weights/
!cp /kaggle/input/siim-covid19-yolov5-weights/{WEIGHTS_FILE} yolov5/{PROJECT_NAME}/{EXP_NAME}/weights/

In [ ]:
import os, pathlib

# This is an inference-only notebook, don't.
WANT_WANDB = False

import gc
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import torch
print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

# 🔨 Prepare Dataset

This is the most important section when it comes to training an object detector with YOLOv5. The directory structure, bounding box format, etc must be in the correct order. This section builds every piece needed to train a YOLOv5 model.

I am using [xhlulu's](https://www.kaggle.com/xhlulu) resized dataset. The uploaded 256x256 Kaggle dataset is [here](https://www.kaggle.com/xhlulu/siim-covid19-resized-to-256px-jpg). Find other image resolutions [here](https://www.kaggle.com/c/siim-covid19-detection/discussion/239918).

* Create train-validation split. <br>
* Create required `/dataset` folder structure and more the images to that folder. <br>
* Create `data.yaml` file needed to train the model. <br>
* Create bounding box coordinates in the required YOLO format. 

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

def convert_dataset():
    image_id = []
    dim0 = []
    dim1 = []
    splits = []
    
    if INFERENCE_ONLY:
        valid_splits = ['test']
    else:
        valid_splits = ['test', 'train']

    # NOTE: For inference only, all you need is test:
    for split in valid_splits:
        save_dir = f'{KAGGLE_RESIZED}/{split}/'

        os.makedirs(save_dir, exist_ok=True)

        for dirname, _, filenames in tqdm(os.walk(f'{KAGGLE_DATASET}/{split}')):
            for file in filenames:
                # set keep_ratio=True to have original aspect ratio
                xray = read_xray(os.path.join(dirname, file))
                im = resize(xray, size=IMG_SIZE)
                im.save(os.path.join(save_dir, file.replace('dcm', 'jpg')))

                image_id.append(file.replace('.dcm', ''))
                dim0.append(xray.shape[0])
                dim1.append(xray.shape[1])
                splits.append(split)

    df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})
    df.to_csv(KAGGLE_RESIZED + '/meta.csv', index=False)


if RUN_ON_ORIGINAL:
    convert_dataset()

In [ ]:
%cd /kaggle/working/yolov5
!python detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --project {PROJECT_NAME} \
                  --name {EXP_NAME} \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf

In [ ]:
PRED_PATH = f'/kaggle/working/yolov5/{PROJECT_NAME}/{EXP_NAME}2/labels'
prediction_files = os.listdir(PRED_PATH)
!ls {PRED_PATH}

print('Number of test images predicted as opaque: ', len(prediction_files))

# Submission

In [ ]:
# Load meta.csv file
# Original dimensions are required to scale the bounding box coordinates appropriately.
meta_df = pd.read_csv(KAGGLE_RESIZED + '/meta.csv')
#meta_df.head(5)

test_meta_df = meta_df.loc[meta_df.split == 'test']
test_meta_df = test_meta_df.drop('split', axis=1)
test_meta_df.columns = ['id', 'dim0', 'dim1']

#test_meta_df.head(5)

In [ ]:
# The submisison requires xmin, ymin, xmax, ymax format. 
# YOLOv5 returns x_center, y_center, width, height
# We also need to rescale the box to the original size!
def correct_bbox_format(bboxes, img_name):
    original_size = test_meta_df.loc[test_meta_df.id == img_name]
    orig_width = original_size['dim0']
    orig_height = original_size['dim1']
    
    correct_bboxes = []
    for b in bboxes:
        xc, yc = int(np.round(b[0]*orig_width)), int(np.round(b[1]*orig_height))
        w, h = int(np.round(b[2]*orig_width)), int(np.round(b[3]*orig_height))

        xmin = xc - int(np.round(w/2))
        xmax = xc + int(np.round(w/2))
        ymin = yc - int(np.round(h/2))
        ymax = yc + int(np.round(h/2))
        
        correct_bboxes.append([xmin, xmax, ymin, ymax])
        
    return correct_bboxes

# Read the txt file generated by YOLOv5 during inference and extract 
# confidence and bounding box coordinates.
def get_conf_bboxes(file_path):
    confidence = []
    bboxes = []
    with open(file_path, 'r') as file:
        for line in file:
            preds = line.strip('\n').split(' ')
            preds = list(map(float, preds))
            confidence.append(preds[-1])
            bboxes.append(preds[1:-1])
    return confidence, bboxes

In [ ]:
# Read the submisison file
sub_df = pd.read_csv(KAGGLE_DATASET + '/sample_submission.csv')
sub_df.tail()

In [ ]:
# Prediction loop for submission
predictions = []

for i in tqdm(range(len(sub_df))):
    row = sub_df.loc[i]
    id_name = row.id.split('_')[0]
    id_level = row.id.split('_')[-1]
    
    if id_level == 'study':
        # do study-level classification
        predictions.append("negative 1 0 0 1 1") # dummy prediction
        
    elif id_level == 'image':
        # we can do image-level classification here.
        # also we can rely on the object detector's classification head.
        # for this example submisison we will use YOLO's classification head. 
        # since we already ran the inference we know which test images belong to opacity.
        if f'{id_name}.txt' in prediction_files:
            # opacity label
            confidence, bboxes = get_conf_bboxes(f'{PRED_PATH}/{id_name}.txt')
            bboxes = correct_bbox_format(bboxes, id_name)
            pred_string = ''
            for j, conf in enumerate(confidence):
                pred_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '
            predictions.append(pred_string[:-1])
        else:
            predictions.append("none 1 0 0 1 1")

In [ ]:
sub_df['PredictionString'] = predictions
sub_df.to_csv('/kaggle/working/submission.csv', index=False)
sub_df.tail()

In [ ]:
# Force nuke all remaining files, as the submission interface gets confused with too many of those:
%cd /kaggle/working
!rm -rf /kaggle/working/yolov5
!rm -rf /kaggle/working/gdcm
!rm -rf /kaggle/working/gdcm.tar